In [1]:
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
from datetime import datetime
from itertools import combinations, permutations
from dateutil.relativedelta import relativedelta
import datetime
from datetime import datetime as dt

In [2]:
# !curl https://raw.githubusercontent.com/CorreaWillian/cointegration/testes/ibov.csv -O

In [3]:
df = pd.read_excel('PORTFOLIO2.xlsx')

In [4]:
df_gpd = df.groupby('ticker').agg({'data_ini': 'min', 'data_fin': 'max', 'setor':'last'})

In [5]:
tickers = df.ticker.unique()

In [6]:
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

In [7]:
df.data_ini.min()

Timestamp('2019-01-01 00:00:00')

In [16]:
utc_from = dt(2019,1,1)
utc_to = dt(2022,7,10)
rates = mt5.copy_rates_range('IBOV', mt5.TIMEFRAME_D1,utc_from, utc_to)
rates_frame = pd.DataFrame(rates)
# convertemos o tempo em segundos no formato datetime
rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
rates_frame['ticker'] = 'IBOV'
rates_frame[['time', 'close', 'ticker']].to_excel('ibov_historico.xlsx')

In [8]:
# Download data
df_list = []
errors = []

utc_from = dt(2019,1,1)
utc_to = dt(2022,7,10)

for i, row in df_gpd.iterrows():
    try: 
        rates = mt5.copy_rates_range(row.name, mt5.TIMEFRAME_D,utc_from, utc_to)

        # criamos a partir dos dados obtidos DataFrame
        rates_frame = pd.DataFrame(rates)
        # convertemos o tempo em segundos no formato datetime
        rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
        rates_frame['ticker'] = row.name

        df_list.append(rates_frame[['time', 'close', 'ticker']])
    except Exception:
        errors.append(row.name)

In [9]:
errors

['ABEV3',
 'ALPA4',
 'AMER3',
 'ASAI3',
 'AZUL4',
 'B3SA3',
 'BBAS3',
 'BBDC3',
 'BBDC4',
 'BBSE3',
 'BEEF3',
 'BIDI11',
 'BPAC11',
 'BPAN4',
 'BRAP4',
 'BRFS3',
 'BRML3',
 'CASH3',
 'CCRO3',
 'CIEL3',
 'CMIG4',
 'CMIN3',
 'COGN3',
 'CPFE3',
 'CPLE6',
 'CRFB3',
 'CSAN3',
 'CSNA3',
 'CVCB3',
 'CYRE3',
 'DXCO3',
 'ECOR3',
 'EGIE3',
 'ELET3',
 'ELET6',
 'EMBR3',
 'ENBR3',
 'ENEV3',
 'ENGI11',
 'EQTL3',
 'EZTC3',
 'FLRY3',
 'GGBR4',
 'GNDI3',
 'GOAU4',
 'GOLL4',
 'HAPV3',
 'HGTX3',
 'HYPE3',
 'IGTI11',
 'IGTI3',
 'IRBR3',
 'ITSA4',
 'ITUB4',
 'JBSS3',
 'JHSF3',
 'KLBN11',
 'LCAM3',
 'LOGG3',
 'LREN3',
 'LWSA3',
 'MGLU3',
 'MRFG3',
 'MRVE3',
 'MULT3',
 'NTCO3',
 'PCAR3',
 'PCAR4',
 'PETR3',
 'PETR4',
 'PETZ3',
 'POSI3',
 'PRIO3',
 'QUAL3',
 'RADL3',
 'RAIL3',
 'RDOR3',
 'RENT3',
 'RRRP3',
 'SANB11',
 'SBSP3',
 'SOMA3',
 'SULA11',
 'SUZB3',
 'TAEE11',
 'TIMS3',
 'TOTS3',
 'UGPA3',
 'USIM5',
 'VALE3',
 'VBBR3',
 'VIIA3',
 'VIVT3',
 'VIVT4',
 'WEGE3',
 'YDUQ3']

In [41]:
df_prices = pd.concat(df_list).pivot(columns='ticker',  index='time', values='close')
df_prices.reset_index(inplace=True)

df_prices['time'] = pd.to_datetime(df_prices['time'])

df_prices['horas_minutos'] = df_prices['time'].dt.time

df_prices = df_prices.loc[df_prices['horas_minutos']>=datetime.time(10, 15)]
df_prices = df_prices.loc[df_prices['horas_minutos']<=datetime.time(16, 45)]


In [42]:
df_prices

ticker,time,ABEV3,ALPA4,AMER3,ASAI3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,...,TOTS3,UGPA3,USIM5,VALE3,VBBR3,VIIA3,VIVT3,WEGE3,YDUQ3,horas_minutos
3,2019-10-01 10:15:00,17.70,25.54,46.89,NaN,50.00,12.95,38.88,20.82,22.71,...,18.69,17.17,6.70,37.64,23.92,8.00,37.15,11.75,34.08,10:15:00
4,2019-10-01 10:20:00,17.73,25.42,46.78,NaN,50.12,12.96,38.85,20.86,22.73,...,18.68,17.16,6.70,37.61,23.99,7.99,37.10,11.76,33.86,10:20:00
5,2019-10-01 10:25:00,17.75,25.46,46.72,NaN,50.08,12.93,38.76,20.89,22.71,...,18.68,17.19,6.68,37.58,23.96,7.98,37.00,11.79,33.83,10:25:00
6,2019-10-01 10:30:00,17.71,25.36,46.82,NaN,49.89,12.86,38.76,20.91,22.76,...,18.71,17.15,6.68,37.73,23.94,7.97,37.08,11.75,33.75,10:30:00
7,2019-10-01 10:35:00,17.77,25.29,46.84,NaN,49.89,12.88,38.87,20.95,22.79,...,18.75,17.17,6.71,37.79,23.99,7.96,37.07,11.78,33.76,10:35:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16882,2020-12-10 10:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:20:00
16883,2021-02-18 10:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:35:00
16884,2021-03-01 11:05:00,NaN,NaN,NaN,14.41,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11:05:00
16885,2021-11-22 10:25:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:25:00


In [43]:
df_prices.to_excel('database5min.xlsx')